In [1]:
from PIL import Image
import glob
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [2]:
labels = pd.read_csv("../Xray_TeethLabels_Simple.csv",index_col=0)
Ylabels = labels[5:]
metaY = labels[:3]
X = pd.DataFrame()
Y = []
for picFilename in glob.glob("../processed2/processed/*"):
    #print(picFilename)
    #print(picFilename.split('/')[3])
    #print(picFilename.split('/')[3].split('_')[1])
    patNumber = picFilename.split('/')[3].split('_')[0]
    toothNumber = picFilename.split('/')[3].split('_')[1].split('.')[0]
    im = Image.open(picFilename)
    X[patNumber + '_' + toothNumber] = np.array(im).flatten()
    Y.append(Ylabels.loc[toothNumber,patNumber])

X_train, X_test, y_train, y_test = train_test_split(X.T.to_numpy(), np.array(Y), test_size=0.20, random_state=100)

In [3]:
print(X.shape)

(750000, 224)


In [4]:
np.random.seed(100)
def svc_param_selection(X, y, nfolds):
    C = [0.1, 1, 10, 100, 1000] #try different values for C
    kernel = ['linear', 'rbf']
    param_grid = {'C': C, 'kernel': kernel}
    grid_search = GridSearchCV(SVC(), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_
best_c = svc_param_selection(X.T.to_numpy(), np.array(Y), 2)
print ("The best value for C is:",best_c)

The best value for C is: {'C': 1, 'kernel': 'rbf'}


In [5]:
model = SVC(C=best_c['C'], kernel=best_c['kernel'])
model.fit(X_train, y_train)
print(model)
# make predictions
expected = y_test
predicted = model.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

SVC(C=1)
              precision    recall  f1-score   support

          No       0.59      0.87      0.70        23
         Yes       0.73      0.36      0.48        22

    accuracy                           0.62        45
   macro avg       0.66      0.62      0.59        45
weighted avg       0.66      0.62      0.60        45

[[20  3]
 [14  8]]


In [6]:
tn, fp, fn, tp = metrics.confusion_matrix(y_test, predicted).ravel()
print(tn)
print(tp)
print(fp)
print(fn)
print(f"sensitivity: {tp/(tp+fn)}")
print(f"Specificity: {tn/(tn+fp)}")
print(f"FPR: {fp/(fp+tn)}")
print(f" FNR: {fn/(fn+tp)}")
print(f"Accuracy: {(tp+tn)/(tp+tn+fp+fn)}")
precision = tp/(tp+fp)
recall = tp/(tp+fn)
print(f"precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {(2*precision*recall)/(precision+recall)}")

20
8
3
14
sensitivity: 0.36363636363636365
Specificity: 0.8695652173913043
FPR: 0.13043478260869565
 FNR: 0.6363636363636364
Accuracy: 0.6222222222222222
precision: 0.7272727272727273
Recall: 0.36363636363636365
F1 score: 0.4848484848484849


In [7]:
import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
import random
import tensorflow as tf

In [8]:
im = Image.open('../processed2/processed/Patient 10_2.png')
print(im)

<PIL.PngImagePlugin.PngImageFile image mode=RGB size=500x500 at 0x7FE4984D1760>


In [9]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
from ipywidgets import FloatProgress

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [10]:
%load_ext autoreload
%autoreload 2
import os,sys
try:
    import lime
except:
    sys.path.append(os.path.join('..', '..')) # add the current directory
    import lime
from lime import lime_image

In [28]:
explainer = lime_image.LimeImageExplainer()

In [38]:
img = tf.reshape(X_train[10], [500, 500,3]).numpy()
print(img.shape)
print(type(img))

(500, 500, 3)
<class 'numpy.ndarray'>


In [73]:
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Dense,Activation

print(X_train.shape)
y_train_new = (y_train=='Yes').astype(int)
X_train_new = tf.reshape(X_train, [179,500,500,3])
print(y_train_new)
print(X_train_new.shape)
#print(X_train_new)

model_keras = Sequential()
model_keras.add(Dense(64, activation='relu'))
model_keras.add(Dense(1, kernel_regularizer=l2(0.01)))
model_keras.add(Activation('linear'))
model_keras.compile(loss='hinge',
              optimizer='adadelta',
              metrics=['accuracy'])

model_keras.fit(X_train_new, y_train_new)

(179, 750000)
[1 0 0 0 1 0 1 1 1 1 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0
 0 1 1 0 1 1 0 0 1 0 0 1 1 1 0 1 0 0 1 1 0 1 0 1 0 1 0 1 0 0 1 1 1 0 1 0 0
 1 1 1 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 0 1 1
 0 1 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0
 0 1 1 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 1 0]
(179, 500, 500, 3)
6/6 [==============================] - 27s 4s/step - loss: 19.2167 - accuracy: 0.4336


In [84]:
%%time
print(X_train[10].shape)
# Hide color is the color for a superpixel turned OFF. Alternatively, if it is NONE, the superpixel will be replaced by the average of its pixels
explanation = explainer.explain_instance(X_train[10], model.predict, hide_color=0, num_samples=1000)

(750000,)


ValueError: Input array must have a shape == (..., 3)), got (1, 750000, 1)

In [11]:
from lime import lime_tabular
classes=['Yes','No']
explainer = lime_tabular.LimeTabularExplainer(X_train,mode='classification',feature_selection= 'auto',
                                               class_names=classes, kernel_width=None,discretize_continuous=True)

In [ ]:
observation_1=10
exp=explainer.explain_instance(X_train[observation_1], model,top_labels=0)
exp.show_in_notebook()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-6fd9a80b633b>", line 2, in <module>
    exp=explainer.explain_instance(X_train[observation_1], model,top_labels=0)
  File "/usr/local/anaconda3/lib/python3.8/site-packages/lime/lime_tabular.py", line 340, in explain_instance
    data, inverse = self.__data_inverse(data_row, num_samples)
  File "/usr/local/anaconda3/lib/python3.8/site-packages/lime/lime_tabular.py", line 550, in __data_inverse
    inverse[1:] = self.discretizer.undiscretize(inverse[1:])
  File "/usr/local/anaconda3/lib/python3.8/site-packages/lime/discretize.py", line 144, in undiscretize
    ret[:, feature] = self.get_undiscretize_values(
  File "/usr/local/anaconda3/lib/python3.8/site-packages/lime/discretize.py", line 127, in get_undiscretize_values
    ret[np.where(min_max_unequal)] = scipy

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-6fd9a80b633b>", line 2, in <module>
    exp=explainer.explain_instance(X_train[observation_1], model,top_labels=0)
  File "/usr/local/anaconda3/lib/python3.8/site-packages/lime/lime_tabular.py", line 340, in explain_instance
    data, inverse = self.__data_inverse(data_row, num_samples)
  File "/usr/local/anaconda3/lib/python3.8/site-packages/lime/lime_tabular.py", line 550, in __data_inverse
    inverse[1:] = self.discretizer.undiscretize(inverse[1:])
  File "/usr/local/anaconda3/lib/python3.8/site-packages/lime/discretize.py", line 144, in undiscretize
    ret[:, feature] = self.get_undiscretize_values(
  File "/usr/local/anaconda3/lib/python3.8/site-packages/lime/discretize.py", line 127, in get_undiscretize_values
    ret[np.where(min_max_unequal)] = scipy

In [12]:
print("test")

test
